# Factors.jl

A factor maps from the cartesion product of its dimensions's supports to a `Float64`.  
`Factor` represent dimensions with a [`Dimensions` datatype](#Dimensions)

`Julia` is column-major, and so are potentials: the first dimension varies over to the first axis (column), the second dimension varies over the second axis (rows) etc ...

In [1]:
using Factors
using DataFrames

In [2]:
ft = Factor([:X, :Y], [1 4; 2 5; 3 6])
DataFrame(ft)

,X,Y,potential
1,1,1,1.0
2,2,1,2.0
3,3,1,3.0
4,1,2,4.0
5,2,2,5.0
6,3,2,6.0


There are multiple constructors:  
The defaults use `Dimensions`:

In [4]:
c = Dimension(:C, 3)
s = Dimension(:S, 10:2:18)

Factor([c, s], rand(3, 5))
Factor(c, [2, 0, 16])

3 instantiations:
	C:  1:3

`Julia` will convert any `<: AbstractVector` to a `Dimension`:

In [5]:
Factor([1 4; 2 5; 3 6], :X => 3:5, :Y => ['a', 'b'])

6 instantiations:
	X:  3:5 (3)
	Y:  ['a','b'] (2)

Or assume the `i`-th `Dimension` is `1:size(potential, i)`

In [6]:
Factor([:X, :Y], rand(20, 16)) 
Factor(:X, [31, 33, 58])

3 instantiations:
	X:  1:3

`Factor`s can also have uniform values (the default is zero), or be uninitialized:

In [ ]:
Factor(c)
Factor(c, 31)
Factor([c, s], nothing) # unitialized
Factor([c, s], 16)
Factor(Dict(:X=>14, :Y=>['Γ', 'Δ'], :Z =>'a':2:'z'))
Factor(Dict(:X=>14, :Y=>['Γ', 'Δ'], :Z =>'a':2:'z'), nothing)
Factor(:A=>10, :B=>3:20)
Factor(16, :A=>10, :B=>3:20) # all 16

A `Factor` can also be zero-dimensional (for weird edge cases):

In [8]:
Factor(2016)

1 instantiation: 2016.0

A `Factor`s scope is its dimensions:

In [15]:
scope(ft)

2-element Array{Factors.Dimension,1}:
 C:  1:3          
 S:  'a':2:'g' (4)

In [9]:
names(ft)

2-element Array{Symbol,1}:
 :X
 :Y

### As an Array

`Factor`s act as `AbstractArray`s in many cases

In [10]:
similar(ft) # unitialized potential

6 instantiations:
	X:  1:3
	Y:  1:2

In [14]:
size(ft, :X)

3

In [15]:
length(ft)

6

In [16]:
ndims(ft)

2

### Indexing

An `Assignment` (or `Pair`s) can index into a `Factor`:

In [21]:
ft = Factor([1 4; 2 5; 3 6], :X => 2:4, :Y=>['a', 'b'])
ft[:X=>[3, 2], :Y=>'a']

2 instantiations:
	X:  [3,2] (2)

In [23]:
ft[Assignment(:X=>[3, 2], :Y=>'a')] = [20, 16]
ft.potential

3×2 Array{Float64,2}:
 16.0  4.0
 20.0  5.0
  3.0  6.0

Besides overloading `sub2ind` and `ind2sub`, functions to convert from and `Assignment`s and assignment tuples are provided:

In [25]:
at2sub(ft, 3, 'b')

(2,2)

In [26]:
sub2at(ft, 1, 1)

(2,'a')

In [ ]:
at2a(ft, 2, 'a')

In [ ]:
a2at(ft, :Y=>'b', :X=>2)

In [ ]:
a2sub(ft, :Y=>'b', :X=>2)

In [ ]:
sub2a(ft, 2, 'a')

### Iterating

Iterating over a factor returns assignment tuples

In [27]:
for t in Factor(:X=>'α':'γ', :Y=>["waldo", "carmen"], :Z=>3)
    println(t)
end

('α',"waldo",1)
('β',"waldo",1)
('γ',"waldo",1)
('α',"carmen",1)
('β',"carmen",1)
('γ',"carmen",1)
('α',"waldo",2)
('β',"waldo",2)
('γ',"waldo",2)
('α',"carmen",2)
('β',"carmen",2)
('γ',"carmen",2)
('α',"waldo",3)
('β',"waldo",3)
('γ',"waldo",3)
('α',"carmen",3)
('β',"carmen",3)


### Patterns

`pattern` returns the sequence of a indices a `Dimension` will take in `Factor` across all indicies

In [47]:
c = Dimension(:C, 3)
s = Dimension(:S, 'a':2:'h')
ft = Factor([c, s])

pattern(ft)

12×2 Array{Int64,2}:
 1  1
 2  1
 3  1
 1  2
 2  2
 3  2
 1  3
 2  3
 3  3
 1  4
 2  4
 3  4

`pattern_states` returns the sequence of states

In [37]:
pattern_states(ft, :C)

3×1 Array{Int64,2}:
 1
 2
 3

This, of course, can be changed by permuting the dimensions:

In [48]:
permutedims!(ft, [2, 1])
pattern(ft)

12×2 Array{Int64,2}:
 1  1
 2  1
 3  1
 4  1
 1  2
 2  2
 3  2
 4  2
 1  3
 2  3
 3  3
 4  3

### Mapping

### Broadcasting

Operations can be broadcast along dimensions:

### Reduce

Dimensions can be reduced.
Convience functions are provded for the following (and their excited cousins, e.g. `sum!`):
* `sum`  
* `prod`  
* `maximum`  
* `minimum`  

### Joining

Factors can be joined through `join` or by multiplying (adding, etc.) them:

### Negatives

By default, negatives are allowed in factors:

In [50]:
Factor(:X, [-2016, 4])

2 instantiations:
	X:  1:2

This can be changed to raise a warning or to throw an error

In [52]:
set_negative_mode(NegativeWarn)
Factor(:X, [-2016, 4])

2 instantiations:
	X:  1:2

In [53]:
set_negative_mode(NegativeError)
Factor(:X, [-2016, 4])

LoadError: ArgumentError: potential has negative values

In [54]:
set_negative_mode(NegativeIgnore)

Factors.NegativeMode{:error}()

## Dimensions

The core unit are dimensions, which are names (`Symbol`) with countably-finite supports (`<: AbstractVector`):

In [13]:
ds = map(s -> Dimension(:X, s), [["bob", "waldo", "superman"], ('a', 'α'), 'a':2:'z', 10:3:40, 2:15, 1:4, 16, []])

8-element Array{Any,1}:
 X:  String["bob","waldo","superman"] (3)
 X:  ['a','α'] (2)                       
 X:  'a':2:'y' (13)                      
 X:  10:3:40 (11)                        
 X:  2:15 (14)                           
 X:  1:4                                 
 X:  1:16                                
 X:  Any[] (0)                           

In [14]:
map(eltype, ds)

8-element Array{Any,1}:
 String
 Char  
 Char  
 Int64 
 Int64 
 Int64 
 Int64 
 Any   

In [15]:
map(spttype, ds)

8-element Array{Any,1}:
 Array{String,1}       
 Array{Char,1}         
 StepRange{Char,Int64} 
 StepRange{Int64,Int64}
 UnitRange{Int64}      
 Base.OneTo{Int64}     
 Base.OneTo{Int64}     
 Array{Any,1}          

### Indexing and iterating

In [56]:
x = Dimension(:X, 'α':'ω')

for v in x
    print(v, " ")
end

α β γ δ ε ζ η θ ι κ λ μ ν ξ ο π ρ ς σ τ υ φ χ ψ ω 

In [57]:
x[2]

'β'

In [58]:
indexin('β', x)

2

In [59]:
(i, d) = update(x, ['α', 'ψ', 'ζ', 'δ'])

([1,24,6,4],X:  ['α','ψ','ζ','δ'] (4))

### Dimension Comparisons

Equality for dimensions is by their support:

In [8]:
Dimension(:X, [1, 2, 3]) == Dimension(:X, 1:1:3)  == Dimension(:X, 1:3) == Dimension(:X, 3)

true

Comparisons use the position of elements in a dimension

In [9]:
o = Dimension(:X, [3, 16, -2])
o .< -2 # here, 3 & 16 are less than -2

3-element BitArray{1}:
  true
  true
 false

In [10]:
# 3, 16, and -2 are all ≥ 3
o .≥ 3

3-element BitArray{1}:
 true
 true
 true